## load model

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
last_checkpoint_34000 = "./third-results/checkpoint-4000"

tokenizer = T5Tokenizer.from_pretrained(last_checkpoint_34000)
model = T5ForConditionalGeneration.from_pretrained(last_checkpoint_34000, device_map="auto")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## load dataset

In [2]:
from datasets import load_dataset


ds = load_dataset("keivalya/MedQuad-MedicalQnADataset")
ds

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

## split the dataset

In [3]:
ds=ds['train'].train_test_split(test_size=0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 14766
    })
    test: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 1641
    })
})

## processing dataset

In [4]:
prefix = "Please answer this medical related question: "

# Define the preprocessing function

def preprocess_function(examples):
   inputs = [prefix + doc for doc in examples["Question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   labels = tokenizer(text_target=examples["Answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 1641/1641 [00:01<00:00, 1475.59 examples/s]


## compute_metrics

In [5]:
import nltk
import evaluate
import numpy as np

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## fine-tuning

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./fourth-results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,  
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=4, 
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=5,
    predict_with_generate=True,
    generation_max_length=200,
    push_to_hub=False
   
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()



c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/5538 [00:10<1:58:16,  1.28s/it]

{'loss': 1.6471, 'grad_norm': 0.6815581321716309, 'learning_rate': 4.99097146984471e-05, 'epoch': 0.01}


  0%|          | 20/5538 [00:20<1:48:17,  1.18s/it]

{'loss': 1.8984, 'grad_norm': 0.8914475440979004, 'learning_rate': 4.981942939689419e-05, 'epoch': 0.01}


  1%|          | 30/5538 [00:34<1:42:32,  1.12s/it]

...

{'loss': 1.7176, 'grad_norm': 1.0609110593795776, 'learning_rate': 3.3477789815817984e-05, 'epoch': 0.99}


 33%|███▎      | 1840/5538 [23:33<43:13,  1.43it/s]

{'loss': 1.8386, 'grad_norm': 0.9416612386703491, 'learning_rate': 3.338750451426508e-05, 'epoch': 1.0}


 33%|███▎      | 1846/5538 [23:40<55:07,  1.12it/s]  

IndexError: piece id is out of range.

## inference

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
last_checkpoint = "./third-results/checkpoint-4000"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint).to("cuda")
finetuned_tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
question="what are marine toxins?"

input_text = "Please answer this medical related question: "+question
input_ids = finetuned_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = finetuned_model.generate(
    input_ids,
    max_length=200,
    min_length=20,
    repetition_penalty=2.0
)
answer = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
from textwrap import fill

print(fill(answer, width=100))

c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Marine toxins are substances that cause damage to the body's tissues and organs. They can be toxic,
but they do not affect other parts of the body. The most common types of marine toxins include:
Lymphadenoma (the type of lymph nodes in the blood) Affected people may have an increased risk for
developing certain diseases such as cancer or heart disease. Some cases of this condition occur when
there is too much fluid in the brain or spinal cord. In some instances, it causes pain, swelling,
loss of appetite, nausea, vomiting, diarrhea, headache, seizures, fatigue, weight gain, muscle
weakness, difficulty swallowing, and/or confusion.
